# ETL_shipment_pro.ipynb
Professional ETL notebook — run all cells

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

DATA_DIR = Path('.')
RAW_CSV = DATA_DIR / 'shipments_raw_demo.csv'
CLEANED_CSV = DATA_DIR / 'cleaned_shipments.csv'
ERRORS_CSV = DATA_DIR / 'errors.csv'
BRANCH_WHITELIST = {'Addis','Dire','Bahir','Hawassa','Adama'}
TRACKING_REGEX = re.compile(r'^AM-\d{3}$')
print('Working dir:', DATA_DIR)
print('Raw CSV:', RAW_CSV)


In [ ]:
def load_raw(path: Path):
    return pd.read_csv(path)

def parse_dates(df):
    for col in ['shipped_at','delivered_at']:
        df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=False)
    return df

def coerce_numeric(df):
    df['weight_kg'] = pd.to_numeric(df['weight_kg'], errors='coerce')
    return df

def clean_text(df):
    df['branch'] = df['branch'].astype(str).str.strip().replace({'': None, 'None': None})
    df['status'] = df['status'].astype(str).str.lower().str.replace(' ', '_')
    return df

def run_validations(df):
    checks = pd.DataFrame(index=df.index)
    checks['tracking_valid'] = df['tracking_number'].astype(str).apply(lambda x: bool(TRACKING_REGEX.match(x)) if pd.notna(x) and str(x)!='None' else False)
    checks['branch_valid'] = df['branch'].isin(BRANCH_WHITELIST)
    checks['ship_date_valid'] = df['shipped_at'].notna()
    checks['delivery_date_valid'] = df['delivered_at'].isna() | (df['delivered_at'] >= df['shipped_at'])
    checks['weight_valid'] = df['weight_kg'].notna() & (df['weight_kg'] > 0)
    checks['duplicate_tracking'] = df.duplicated(['tracking_number'], keep=False)
    return checks

def split_errors(df, checks):
    critical = ~(checks['tracking_valid'] & checks['ship_date_valid'] & checks['weight_valid'] & checks['delivery_date_valid'])
    errors = df[critical].copy()
    cleaned = df[~critical].copy()
    return cleaned, errors

def apply_cleaning(cleaned):
    cleaned = cleaned.copy()
    cleaned['branch'] = cleaned['branch'].fillna('Unknown')
    cleaned['transit_days'] = (cleaned['delivered_at'] - cleaned['shipped_at']).dt.days
    return cleaned


In [ ]:
raw = load_raw(RAW_CSV)
print('Loaded rows:', len(raw))
raw.head()

In [ ]:
df = raw.copy()
df = df.pipe(clean_text).pipe(parse_dates).pipe(coerce_numeric)
checks = run_validations(df)
print('Failures per rule:')
print(checks.apply(lambda s: (~s).sum()))

In [ ]:
cleaned, errors = split_errors(df, checks)
cleaned = apply_cleaning(cleaned)
print('Cleaned rows:', len(cleaned), 'Error rows:', len(errors))
cleaned.head()

In [ ]:
assert cleaned['tracking_number'].notna().all(), 'Null tracking numbers in cleaned data!'
assert cleaned['branch'].isin(BRANCH_WHITELIST | {'Unknown'}).all(), 'Unexpected branch label in cleaned data!'
assert (cleaned['weight_kg'] > 0).all(), 'Non-positive weight detected in cleaned data!'
assert not cleaned.duplicated(['tracking_number']).any(), 'Duplicate tracking numbers remain in cleaned data!'
print('All assertions passed.')

In [ ]:
cleaned.to_csv(CLEANED_CSV, index=False)
errors.to_csv(ERRORS_CSV, index=False)
print('Saved cleaned to', CLEANED_CSV)
print('Saved errors to', ERRORS_CSV)
